In [1]:
import re
import numpy as np
import pandas as pd

import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
cc_data = pd.read_csv('creditcard.csv')

In [3]:
cc_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [7]:
cc_data.loc[cc_data['Class']==1]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [10]:
cc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [12]:
cc_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [16]:
cc_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

### very unbalanced Dataset

In [17]:
legit = cc_data[cc_data.Class == 0]
fraud = cc_data[cc_data.Class == 1]

In [18]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [19]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [20]:
cc_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Under Sampling

Building sample dataset containing similar distribution of normal and fraud transactions

In [21]:
legit_sample = legit.sample(n=492)

In [22]:
cc_data_new = pd.concat([legit_sample, fraud], axis=0)

In [23]:
cc_data_new.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
162783,115364.0,-0.194448,0.753427,-1.046363,-1.214643,0.583129,-0.863471,0.885768,-0.110569,-1.471863,...,-0.094599,0.484857,-0.210221,-0.490538,-0.444565,-0.178662,0.052952,-0.010717,4.73,0
224738,143940.0,2.312020,-1.192680,-1.418972,-1.767833,-0.773811,-0.739782,-0.793853,-0.292495,-1.924670,...,0.110650,0.819924,0.004714,-0.428686,0.125128,0.078221,-0.009735,-0.073950,15.00,0
51356,44908.0,1.190850,0.013254,-0.131428,1.468640,1.746781,4.321727,-1.132818,1.133232,-0.135110,...,0.049967,0.112904,-0.080523,1.012668,0.553756,0.122172,0.041327,0.027239,0.76,0
96757,65923.0,-2.145640,-0.273959,2.099973,-1.717589,-1.033955,0.565512,-0.533682,-0.630533,-0.447509,...,1.085614,0.965200,-0.293294,0.054769,0.327208,-0.207648,-0.838176,-0.291156,85.00,0
108145,70785.0,1.239730,-1.468991,0.669442,-1.276237,-1.752083,-0.290353,-1.172860,-0.039334,-1.634970,...,0.036439,0.308070,-0.070846,0.088534,0.231742,-0.086446,0.048202,0.043730,124.00,0


In [24]:
cc_data_new = cc_data_new.sample(frac=1, random_state=1).reset_index(drop=True)

In [25]:
cc_data_new.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,155542.0,1.868226,1.363077,-1.994934,4.173516,1.239751,-0.746186,0.572731,-0.131235,-1.551839,...,-0.301001,-0.818972,0.206812,-0.263683,-0.114958,-0.240672,-0.006629,0.017258,3.14,1
1,113633.0,2.132304,-0.029822,-1.747089,0.012558,0.569675,-0.505652,0.165392,-0.261383,0.766815,...,0.219633,0.795932,-0.078317,0.062030,0.456409,-0.410220,0.004148,-0.053725,1.00,0
2,126147.0,0.254793,1.653727,-2.119789,-0.200879,2.160636,-2.111508,2.089224,-0.881253,-0.472694,...,0.040441,0.521838,-0.324952,-0.257498,-0.820900,0.021359,-0.215590,-0.123954,0.77,0
3,93879.0,-13.086519,7.352148,-18.256576,10.648505,-11.731476,-3.659167,-14.873658,8.810473,-5.418204,...,2.761157,-0.266162,-0.412861,0.519952,-0.743909,-0.167808,-2.498300,-0.711066,30.31,1
4,68357.0,1.232604,-0.548931,1.087873,0.894082,-1.433055,-0.356797,-0.717492,0.003167,-0.100397,...,-0.448671,-0.517568,0.012833,0.699217,0.527258,-0.322607,0.080805,0.035427,19.59,1


In [27]:
cc_data_new['Class'].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [28]:
cc_data_new.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94226.743902,-0.056289,-0.078987,0.001115,-0.089655,0.067074,0.032177,0.064223,0.030253,-0.048649,...,-0.033602,-0.009142,-0.020860,0.020689,-0.021556,-0.023932,-0.012713,-0.046658,-0.010967,90.822419
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [29]:
y = cc_data_new['Class']
x = cc_data_new.drop(columns='Class', axis=1)

In [30]:
x

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,155542.0,1.868226,1.363077,-1.994934,4.173516,1.239751,-0.746186,0.572731,-0.131235,-1.551839,...,-0.172216,-0.301001,-0.818972,0.206812,-0.263683,-0.114958,-0.240672,-0.006629,0.017258,3.14
1,113633.0,2.132304,-0.029822,-1.747089,0.012558,0.569675,-0.505652,0.165392,-0.261383,0.766815,...,-0.188730,0.219633,0.795932,-0.078317,0.062030,0.456409,-0.410220,0.004148,-0.053725,1.00
2,126147.0,0.254793,1.653727,-2.119789,-0.200879,2.160636,-2.111508,2.089224,-0.881253,-0.472694,...,0.035066,0.040441,0.521838,-0.324952,-0.257498,-0.820900,0.021359,-0.215590,-0.123954,0.77
3,93879.0,-13.086519,7.352148,-18.256576,10.648505,-11.731476,-3.659167,-14.873658,8.810473,-5.418204,...,-1.376298,2.761157,-0.266162,-0.412861,0.519952,-0.743909,-0.167808,-2.498300,-0.711066,30.31
4,68357.0,1.232604,-0.548931,1.087873,0.894082,-1.433055,-0.356797,-0.717492,0.003167,-0.100397,...,-0.576274,-0.448671,-0.517568,0.012833,0.699217,0.527258,-0.322607,0.080805,0.035427,19.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,85285.0,-6.713407,3.921104,-9.746678,5.148263,-5.151563,-2.099389,-5.937767,3.578780,-4.684952,...,0.135711,0.954272,-0.451086,0.127214,-0.339450,0.394096,1.075295,1.649906,-0.394905,252.92
980,73528.0,-1.959144,1.642899,0.575369,-1.385062,-0.304486,-0.803276,0.243575,0.451949,-0.016911,...,-0.090251,0.035682,0.095051,0.090510,0.150452,-0.274250,0.723588,-0.552252,-0.126796,3.76
981,144839.0,-6.423306,1.658515,-5.866440,2.052064,-0.615817,-3.372266,-5.036556,2.643106,-2.274630,...,-0.713516,0.641211,-0.256678,-2.337233,-0.158278,1.198797,-0.261258,0.780125,-0.731801,0.00
982,130292.0,1.988472,-0.336969,-0.416633,0.381683,-0.398207,-0.101313,-0.568461,-0.033284,1.274540,...,-0.100635,0.196601,0.841046,0.122523,0.721027,-0.020902,-0.246518,0.044259,-0.027189,12.99


In [31]:
y

0      1
1      0
2      0
3      1
4      1
      ..
979    1
980    0
981    1
982    0
983    0
Name: Class, Length: 984, dtype: int64

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state =3)

In [39]:
LRmodel = LogisticRegression(C = 1, max_iter = 1000, n_jobs=-1)

In [42]:
LRmodel = LogisticRegression()

In [43]:
LRmodel.fit(x_train, y_train)

LogisticRegression()

In [44]:
y_pred = LRmodel.predict(x_test)
# target_names = ["Negative","Positive"]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95        97
           1       0.95      0.96      0.96       100

    accuracy                           0.95       197
   macro avg       0.95      0.95      0.95       197
weighted avg       0.95      0.95      0.95       197

